# Socio-Temporal Risk Analysis & Prediction. (falta resumen
)

In [ ]:
# ==============================================================================
# PIPELINE DE ENTRENAMIENTO: ARQUITECTURA  (V4.1)FINAL
# PROPOSITO: Entrenamiento de modelo predictivo basado en Bosques Aleatorios (Random Forest)
# integrando componentes autorregresivos (Time-Series) y regresores exógenos (Socioeconomía).
# ==============================================================================

import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# ------------------------------------------------------------------------------
# 1. MODULO DE INGESTA Y NORMALIZACIÓN DE DATOS (ETL)
# ------------------------------------------------------------------------------
# Objetivo: Cargar fuentes heterogéneas y asegurar la integridad referencial para el cruce de datos.

print("[INFO] Iniciando ejecucion del pipeline de entrenamiento V4.1...")

# Carga de datasets desde rutas locales definidas
# Se utilizan rutas absolutas para garantizar la localización de los insumos en el entorno de despliegue.
ruta_dataset_crimen = 'C:/Users/Oliver/Downloads/demo1/src/Dataset_Homicidios_Panama_2017_2024_NormalizadoFINAL.xlsx'
ruta_dataset_contexto = 'C:/Users/Oliver/Downloads/demo1/src/Datos_Contexto_Anual_MEJORADO.csv'

df_crimen = pd.read_excel(ruta_dataset_crimen)
df_contexto = pd.read_csv(ruta_dataset_contexto)

# Estandarización de claves primarias (Primary Key Normalization)
# Se aplica limpieza de cadenas (trim/upper) para garantizar que el JOIN entre datasets no pierda registros.
df_crimen['PROVINCIA'] = df_crimen['PROVINCIA'].astype(str).str.upper().str.strip()
df_contexto['PROVINCIA'] = df_contexto['PROVINCIA'].astype(str).str.upper().str.strip()

print("[INFO] Ejecutando fusion de datos (Data Enrichment)...")

# Enriquecimiento del Dataset
# Se utiliza un LEFT JOIN para anexar las variables socioeconómicas anuales a cada registro criminal.
df_full = pd.merge(df_crimen, df_contexto, on=['AÑO', 'PROVINCIA'], how='left')
df_full = df_full.fillna(0)  # Manejo de valores nulos mediante imputación a cero (Zero Imputation)

# ------------------------------------------------------------------------------
# 2. INGENIERÍA DE CARACTERÍSTICAS (FEATURE ENGINEERING)
# ------------------------------------------------------------------------------
# Objetivo: Transformar datos crudos en vectores matemáticos densos que capturen 
# patrones temporales (Lags) y estructurales.

print("[INFO] Generando estructuras temporales y calculo de volatilidad...")

# Reducción de Dimensionalidad (Aggregation)
# Se agrupan los incidentes individuales para obtener una serie temporal discreta (Mes/Provincia).
if 'contador' not in df_full.columns: df_full['contador'] = 1

df_ml = df_full.groupby(['AÑO', 'MES', 'PROVINCIA']).agg({
    'contador': 'sum',              # Variable Objetivo (Target): Volumen de criminalidad
    'POBLACION_ESTIMADA': 'first',  # Variable Exógena Demográfica
    'TASA_DESEMPLEO': 'first',      # Variable Exógena Económica
    'INDICE_PANDILLAS': 'first'     # Variable Exógena de Seguridad
}).reset_index()

df_ml.rename(columns={'contador': 'TOTAL_HOMICIDIOS'}, inplace=True)

# Transformación de Variable Temporal (Ordinal Encoding)
# Convierte la representación textual de los meses a valores numéricos ordinales para preservar la secuencia temporal.
mapa_meses = {'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
              'JULIO': 7, 'AGOSTO': 8, 'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12}

df_ml['MES'] = df_ml['MES'].astype(str).str.upper().str.strip()
df_ml['MES_NUM'] = df_ml['MES'].map(mapa_meses)
df_ml = df_ml.dropna(subset=['MES_NUM']) # Eliminación de registros con formato de fecha inválido

# Codificación de Variables Categóricas (Label Encoding)
# Transforma la variable nominal 'PROVINCIA' en códigos numéricos interpretables por el algoritmo.
df_ml['PROVINCIA_CODE'] = df_ml['PROVINCIA'].astype('category').cat.codes

# Ordenamiento cronológico estricto
# Requerido para que las funciones de ventana (Rolling Windows) calculen correctamente el pasado.
df_ml = df_ml.sort_values(by=['PROVINCIA', 'AÑO', 'MES_NUM'])

# --- CONSTRUCCIÓN DE VARIABLES DE SERIES DE TIEMPO (LAG FEATURES) ---

# 1. Autocorrelación de Primer Orden (Lag-1)
# Captura la inercia inmediata: La correlación entre los eventos del mes actual y el mes previo.
df_ml['HOMICIDIOS_MES_ANTERIOR'] = df_ml.groupby('PROVINCIA')['TOTAL_HOMICIDIOS'].shift(1)

# 2. Tendencia Trimestral (Rolling Mean)
# Suaviza el ruido mensual para identificar la tendencia subyacente de los últimos 3 meses (Moving Average).
df_ml['PROMEDIO_TRIMESTRE'] = df_ml.groupby('PROVINCIA')['TOTAL_HOMICIDIOS'].transform(lambda x: x.rolling(3).mean().shift(1))

# 3. Métrica de Volatilidad Estocástica (Rolling Std - NUEVO EN V4.0)
# Calcula la desviación estándar móvil. Permite al modelo cuantificar la "estabilidad" o "caos" de la zona.
# Zonas con alta volatilidad penalizan la confianza de la predicción puntual.
df_ml['VOLATILIDAD_TRIMESTRE'] = df_ml.groupby('PROVINCIA')['TOTAL_HOMICIDIOS'].transform(lambda x: x.rolling(3).std().shift(1))

# Limpieza final de vectores
# Se imputan los primeros meses de la serie (que quedan vacíos por el desplazamiento) para no perder datos de entrenamiento.
df_ml = df_ml.fillna(0)

# ------------------------------------------------------------------------------
# 3. ENTRENAMIENTO Y OPTIMIZACIÓN DEL MODELO
# ------------------------------------------------------------------------------
# Algoritmo: Random Forest Regressor
# Justificación: Modelo de ensamblaje no paramétrico, robusto a outliers y capaz de capturar relaciones no lineales.

# Definición del Vector de Características (Feature Space - 9 Dimensiones)
features = [
    'AÑO', 'MES_NUM', 'PROVINCIA_CODE',         # Componentes Temporales/Espaciales
    'HOMICIDIOS_MES_ANTERIOR',                  # Componente Autoregresivo (Lag)
    'PROMEDIO_TRIMESTRE',                       # Componente de Tendencia
    'VOLATILIDAD_TRIMESTRE',                    # Componente de Riesgo/Varianza
    'POBLACION_ESTIMADA', 'TASA_DESEMPLEO', 'INDICE_PANDILLAS' # Regresores Socioeconómicos
]

X = df_ml[features]
y = df_ml['TOTAL_HOMICIDIOS']

# Partición de Datos (Hold-out Validation)
# Se reserva un 20% de los datos para validación externa y detección de overfitting.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"[INFO] Iniciando ajuste de hiperparametros con {X.shape[1]} variables predictoras...")

# Configuración del Estimador con Regularización
modelo_ultra = RandomForestRegressor(
    n_estimators=300,       # Aumentado a 300 para reducir la varianza del error (Teorema del Límite Central)
    min_samples_split=2,
    min_samples_leaf=4,     # Restricción: Cada hoja debe tener al menos 4 muestras (Suavizado de predicciones)
    max_features='log2',    # Selección aleatoria de features para descorrelacionar los árboles
    max_depth=None,
    ccp_alpha=0.015,        # Cost-Complexity Pruning: Poda estadística para eliminar ramas ruidosas
    bootstrap=True,
    random_state=42,        # Semilla para reproducibilidad científica
    n_jobs=-1               # Paralelización total en CPU
)

modelo_ultra.fit(X_train, y_train)

# ------------------------------------------------------------------------------
# 4. EVALUACIÓN DE DESEMPEÑO Y DESPLIEGUE
# ------------------------------------------------------------------------------
# Inferencia sobre el conjunto de prueba
predicciones = modelo_ultra.predict(X_test)

# Cálculo de Métricas de Bondad de Ajuste
r2 = r2_score(y_test, predicciones)
mae = mean_absolute_error(y_test, predicciones)

print(f"""
[REPORTE] METRICAS DE RENDIMIENTO (V4.1 - Dataset Enriquecido):
---------------------------------------------------------------
[METRICA] Coeficiente de Determinacion (R2): {r2:.2%}
          -> Interpretacion: El modelo explica el {r2:.1%} de la varianza en los datos criminales.

[METRICA] Error Absoluto Medio (MAE):        {mae:.4f}
          -> Interpretacion: Desviacion promedio de {mae:.2f} incidentes respecto a la realidad.

[INFO] Configuracion Final: 9 Variables (Incluye Volatilidad Estocastica).
""")

# Serialización del Objeto (Marshalling)
# Se persiste el modelo entrenado en disco para su posterior consumo via API/Frontend.
nombre_archivo = 'modelo_homicidios_panama_socioeconomico_ULTRA.pkl'
joblib.dump(modelo_ultra, nombre_archivo)
print(f"[EXITO] Modelo serializado y exportado correctamente en: {nombre_archivo}")

[INFO] Iniciando ejecucion del pipeline de entrenamiento V4.1...
[INFO] Ejecutando fusion de datos (Data Enrichment)...
[INFO] Generando estructuras temporales y calculo de volatilidad...
[INFO] Iniciando ajuste de hiperparametros con 9 variables predictoras...

[REPORTE] METRICAS DE RENDIMIENTO (V4.1 - Dataset Enriquecido):
---------------------------------------------------------------
[METRICA] Coeficiente de Determinacion (R2): 76.41%
          -> Interpretacion: El modelo explica el 76.4% de la varianza en los datos criminales.

[METRICA] Error Absoluto Medio (MAE):        1.9673
          -> Interpretacion: Desviacion promedio de 1.97 incidentes respecto a la realidad.

[INFO] Configuracion Final: 9 Variables (Incluye Volatilidad Estocastica).

[EXITO] Modelo serializado y exportado correctamente en: modelo_homicidios_panama_socioeconomico_ULTRA.pkl


Es importante señalar que las métricas de desempeño del modelo pueden presentar ligeras fluctuaciones controladas debido a la naturaleza estocástica del proceso de entrenamiento y generación de datos. Al incorporar ruido estadístico intencional en las variables socioeconómicas (como la actividad de pandillas), se simula la incertidumbre inherente a los fenómenos criminales del mundo real. Esta variabilidad garantiza que el sistema no se sobreajuste (overfitting) a un escenario estático, priorizando la robustez operativa y la capacidad de adaptación ante entornos dinámicos sobre una precisión teórica artificial